In [ ]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO
import time
from datetime import datetime


model = YOLO("yolo11n.pt")
output_file = "person_count.txt"
with open(output_file, "w") as file:
    file.write("Timestamp, Frame Number, Person Count\n") 

PERSON_CLASS_ID = 0
start_time = time.time()
last_log_time = start_time 

video_path = "Metro/Crowding Near Ticketing line_1.avi"
cap = cv2.VideoCapture(video_path)


track_history = defaultdict(lambda: [])

while cap.isOpened():
  
    success, frame = cap.read()

    if not success:
        break 

   
    results = model.track(frame, persist=True)

    try:
        boxes = results[0].boxes.xywh.cpu().numpy()  # Bounding boxes in xywh format
        class_ids = results[0].boxes.cls.cpu().numpy().astype(int)  # Class labels
        track_ids = results[0].boxes.id.cpu().numpy().astype(int)  # Track IDs

        # Filter detections for "person" class only
        person_indices = [i for i, cls_id in enumerate(class_ids) if cls_id == PERSON_CLASS_ID]
        person_count = len(person_indices)
        person_boxes = boxes[person_indices]
        person_track_ids = [track_ids[i] for i in person_indices]
        # Write to the file approximately every 1 minute
        current_time = time.time()
        if current_time - last_log_time >= 20:  # Check if 1 minute has elapsed
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            with open(output_file, "a") as file:
                file.write(f"{timestamp},  {person_count}\n")
            last_log_time = current_time 

        # Annotate the frame with only "person" class detections
        annotated_frame = frame.copy()

        for box, track_id in zip(person_boxes, person_track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((x, y))  # Add center point of the detection to the track history

            #Max Age
            if len(track) > 100:
                track.pop(0)

            # Draw the bounding box and track ID
            x1, y1, x2, y2 = int(x - w / 2), int(y - h / 2), int(x + w / 2), int(y + h / 2)
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(annotated_frame, f"ID: {track_id}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            # Draw tracking lines
            # points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
            # cv2.polylines(annotated_frame, [points], isClosed=False, color=(0, 255, 255), thickness=2)

        cv2.imshow("YOLO11 Tracking - Person", annotated_frame)

    except IndexError:
        print("No detections in this frame.")

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()
